In [1]:
library(RPostgreSQL)
library(twang)
library(tidyverse)

Loading required package: DBI
Loading required package: gbm
Loading required package: survival
Loading required package: lattice
Loading required package: splines
Loading required package: parallel
Loaded gbm 2.1.3
Loading required package: survey
Loading required package: grid
Loading required package: Matrix

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

Loading required package: xtable
Loading required package: latticeExtra
Loading required package: RColorBrewer
Loading required package: MASS
## 
##  Matching (Version 4.9-2, Build Date: 2015-12-25)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##

── Attaching packages ─────────────────────────────────────── tidyverse 1

In [2]:
data_dir <- file.path("..", "data")
sql_dir <- file.path("..", "sql")

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [4]:
full_data <- dbGetQuery(con, "select * from merged_data")
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_sodium_flag,lab_bun_flag,lab_bicarbonate_flag,lab_bnp_flag,lab_pco2_flag,lab_creatinine_flag,lab_potassium_flag,lab_troponin_flag,lab_po2_flag,lab_lactate_flag
228416,159895,5491,MICU,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,1,2151-06-30,⋯,1,1,1,0,1,1,1,0,1,1
278148,169700,49395,MICU,2197-01-09 00:45:42,2197-01-10 06:46:29,1,82.72606,1,NA,⋯,1,1,1,0,0,1,1,0,0,0
274174,103722,14855,MICU,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,1,2181-09-11,⋯,1,1,1,0,1,1,1,0,1,1
214954,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,1,1,0,1,1,1,1,1,1
222457,183493,96815,MICU,2105-06-04 20:57:55,2105-06-06 20:01:51,1,62.84018,1,NA,⋯,1,1,1,0,0,1,1,0,0,0
264413,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,1,1,1,0,0,1,1,0,0,1


In [5]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE

In [6]:
to_factor <- function(x) {
    if(length(na.omit(unique(x))) <= 1) return(factor(x, levels = c(0, 1)))
    return(factor(x))
}

In [7]:
factor_vars <- full_data %>%
    names %>%
    grep("flag|abnormal|icd|sedative", ., value = TRUE) %>%
    c("gender", "first_careunit", "echo", "vent", "vaso",
      "icu_adm_hour", "mort_28_day")
factor_vars

[1] "icd_chf"                        "icd_afib"                      
 [3] "icd_renal"                      "icd_liver"                     
 [5] "icd_copd"                       "icd_cad"                       
 [7] "icd_stroke"                     "icd_malignancy"                
 [9] "lab_hemoglobin_abnormal"        "lab_platelet_abnormal"         
[11] "lab_creatinine_kinase_abnormal" "lab_wbc_abnormal"              
[13] "lab_ph_abnormal"                "lab_chloride_abnormal"         
[15] "lab_sodium_abnormal"            "lab_bun_abnormal"              
[17] "lab_bicarbonate_abnormal"       "lab_bnp_abnormal"              
[19] "lab_pco2_abnormal"              "lab_creatinine_abnormal"       
[21] "lab_potassium_abnormal"         "lab_troponin_abnormal"         
[23] "lab_po2_abnormal"               "lab_lactate_abnormal"          
[25] "sedative"                       "vs_heart_rate_flag"            
[27] "vs_cvp_flag"                    "vs_map_flag"                   
[29] "vs_temp_flag"                   "lab_hemoglobin_flag"           
[31] "lab_platelet_flag"              "lab_creatinine_kinase_flag"    
[33] "lab_wbc_flag"                   "lab_ph_flag"                   
[35] "lab_chloride_flag"              "lab_sodium_flag"               
[37] "lab_bun_flag"                   "lab_bicarbonate_flag"          
[39] "lab_bnp_flag"                   "lab_pco2_flag"                 
[41] "lab_creatinine_flag"            "lab_potassium_flag"            
[43] "lab_troponin_flag"              "lab_po2_flag"                  
[45] "lab_lactate_flag"               "gender"                        
[47] "first_careunit"                 "echo"                          
[49] "vent"                           "vaso"                          
[51] "icu_adm_hour"                   "mort_28_day"

In [8]:
weekday <- c("sun", "mon", "tues", "wednes", "thurs", "fri", "satur") %>%
paste0("day")
weekday

[1] "sunday"    "monday"    "tuesday"   "wednesday" "thursday"  "friday"   
[7] "saturday"

In [9]:
full_data <- full_data %>%
    mutate(echo_int = as.integer(echo)) %>%
    mutate(icu_adm_weekday = weekday[icu_adm_weekday + 1]) %>%
    mutate(mort_28_day_int = as.integer(mort_28_day)) %>%
    mutate_at(factor_vars, to_factor) %>%
    mutate(icu_adm_weekday = factor(icu_adm_weekday, levels = weekday)) %>%
    mutate(gender = relevel(gender, "M")) %>%
    mutate(first_careunit = relevel(first_careunit, "SICU"))
full_data %>% pull(echo) %>% head

[1] 1 0 1 1 0 1
Levels: 0 1

In [10]:
feature_names <- full_data %>%
    names %>%
    keep(grepl("vs|lab|icd|age|gender|weight|saps|sofa|elix_score|vent|vaso|icu_adm|careunit|sedative", .)) %>%
    discard(grepl("vs|lab", .) & grepl("flag", .) & !grepl("bnp|troponin|kinase|cvp", .)) %>%
    discard(grepl("bnp|troponin|kinase|cvp", .) & !grepl("flag", .)) %>%
    discard(grepl("min|max", .)) %>%
    discard(grepl("abnormal", .))
feature_names
length(feature_names)

[1] "first_careunit"             "age"                       
 [3] "gender"                     "weight"                    
 [5] "saps"                       "sofa"                      
 [7] "elix_score"                 "vent"                      
 [9] "vaso"                       "icu_adm_weekday"           
[11] "icu_adm_hour"               "icd_chf"                   
[13] "icd_afib"                   "icd_renal"                 
[15] "icd_liver"                  "icd_copd"                  
[17] "icd_cad"                    "icd_stroke"                
[19] "icd_malignancy"             "vs_heart_rate_first"       
[21] "vs_map_first"               "vs_temp_first"             
[23] "lab_hemoglobin_first"       "lab_platelet_first"        
[25] "lab_wbc_first"              "lab_ph_first"              
[27] "lab_chloride_first"         "lab_sodium_first"          
[29] "lab_bun_first"              "lab_bicarbonate_first"     
[31] "lab_pco2_first"             "lab_creatinine_first"      
[33] "lab_potassium_first"        "lab_po2_first"             
[35] "lab_lactate_first"          "sedative"                  
[37] "vs_cvp_flag"                "lab_creatinine_kinase_flag"
[39] "lab_bnp_flag"               "lab_troponin_flag"

[1] 40

In [11]:
feature_names %>%
data.frame(feature = ., stringsAsFactors = FALSE) %>%
data.table::fwrite(file = file.path(data_dir, "features.csv"))